In [ ]:
import numpy as np  
import pandas as pd
df = pd.read_csv('/kaggle/input/ashrae-energy-prediction/train.csv')
building = pd.read_csv('/kaggle/input/ashrae-energy-prediction/building_metadata.csv')
weather = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_train.csv')
building.loc[:,'primary_use'] = building['primary_use'].replace({'Entertainment/public assembly':'public',
     'Public services':'service','Lodging/residential':'resident','Healthcare':'resident',
     'Parking':'parking','Warehouse/storage': 'parking','Manufacturing/industrial': 'manufact',
     'Retail':'service', 'Services':'service', 'Technology/science':'Office',
     'Food sales and service':'service','Utility':'service','Religious worship':'public'})
usage = building['primary_use'].unique()
building.loc[:,'primary_use'] = building['primary_use'].replace(dict(zip(usage,range(len(usage)))))

df = df.join(building, on='building_id', lsuffix='_1', rsuffix='_2')
df.loc[:,'square_feet'] = df['square_feet'].apply(np.log1p)
df.loc[:,'floor_count'] = df['floor_count'].fillna(df['floor_count'].mean())
df.loc[:,'year_built'] = df['year_built'].fillna(df['year_built'].median())
def process_year(x):
     return 1- (x-1900)/(2017-1900)
        
df.loc[:,'year_built'] = df['year_built'].apply(process_year)

for col in ['meter_reading','square_feet','year_built','floor_count']:
     df[col] = df[col].astype(np.float16)

for col in ['building_id_1','meter','floor_count']:
     df[col] = df[col].astype(np.int16)

df.pop('building_id_2')    

df = df.join(weather, lsuffix='_1', rsuffix='_2')
df = df[['building_id_1', 'meter', 'site_id_1', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 
       'sea_level_pressure', 'wind_direction', 'wind_speed','meter_reading']]
df.fillna(0, inplace=True)
for col in ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr',
            'sea_level_pressure','wind_direction','wind_speed'] :
       df[col] = df[col].astype(np.float16)

df.to_csv('dump.csv',index=False)

In [ ]:
df = pd.read_csv('dump.csv')
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import statistics

X, y = df[df.columns[:-1]], df['meter_reading']
   
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
model = xgb.XGBRegressor(n_estimators=10,max_depth=10,reg_lambda=0.5)

model.fit(X_train, y_train, early_stopping_rounds=50, eval_set=[(X_val, y_val)], verbose=True)
y_prediction = model.predict(X_val)

print(mean_squared_error(y_val,y_prediction))

In [ ]:
X

In [ ]:
for i in X.columns:
    print(X[X[i].isna()])